In [9]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
x1_vals_path = os.path.join('x1_vals.json')
x2_vals_path = os.path.join('x2_vals.json')
dummy_x1_vals_path = os.path.join('shared/dummy_x1_vals.json')
dummy_x2_vals_path = os.path.join('shared/dummy_x2_vals.json')

x1_vals= np.array(json.loads(open(x1_vals_path, "r").read())['input_data'][0])
dummy_x1_vals = np.random.uniform(min(x1_vals), max(x1_vals), len(x1_vals))
json.dump({"input_data":[dummy_x1_vals.tolist()]}, open(dummy_x1_vals_path, 'w'))

x2_vals= np.array(json.loads(open(x2_vals_path, "r").read())['input_data'][0])
dummy_x2_vals = np.random.uniform(min(x2_vals), max(x2_vals), len(x2_vals))
json.dump({"input_data":[dummy_x2_vals.tolist()]}, open(dummy_x2_vals_path, 'w'))


y_vals_path = os.path.join('y_vals.json')
dummy_y_vals_path = os.path.join('shared/dummy_y_vals.json')

y_vals= np.array(json.loads(open(y_vals_path, "r").read())['input_data'][0])
dummy_y_vals = np.random.uniform(min(y_vals), max(y_vals), len(y_vals))
json.dump({"input_data":[dummy_y_vals.tolist()]}, open(dummy_y_vals_path, 'w'))


def stacked_x(*args):
    result = np.column_stack((*args, np.ones_like(args[0])))
    return result

In [75]:

# where y value > 20.0
lt20_y_vals = y_vals[y_vals<20.0]
lt20_y_vals_tensor = torch.tensor(lt20_y_vals).reshape(1,-1,1)
new_size = torch.tensor(lt20_y_vals_tensor.size()[1])

lt20_dummy_y_vals = dummy_y_vals[dummy_y_vals<20.0]
print("lennn: ", len(lt20_dummy_y_vals))
lt20_dummy_y_vals_tensor = torch.tensor(lt20_dummy_y_vals).reshape(1,-1,1)
new_dummy_size = torch.tensor(lt20_dummy_y_vals_tensor.size()[1])

x_one = stacked_x(x1_vals, x2_vals)
lt20_x_one = stacked_x(x1_vals[y_vals<20.0], x2_vals[y_vals<20.0])
lt20_x_one_tensor = torch.tensor(lt20_x_one).reshape(1, len(lt20_x_one), -1)


dummy_x_one = stacked_x(dummy_x1_vals, dummy_x2_vals)
lt20_dummy_x_one = stacked_x(dummy_x1_vals[dummy_y_vals<20.0], dummy_x2_vals[dummy_y_vals<20.0])
lt20_dummy_x_one_tensor = torch.tensor(lt20_dummy_x_one).reshape(1, len(lt20_dummy_x_one), -1)

w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_one.transpose(), x_one)), x_one.transpose()), y_vals)
lt20_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(lt20_x_one.transpose(), lt20_x_one)), lt20_x_one.transpose()), lt20_y_vals)
dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(dummy_x_one.transpose(), dummy_x_one)), dummy_x_one.transpose()), dummy_y_vals)
lt20_dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(lt20_dummy_x_one.transpose(), lt20_dummy_x_one)), lt20_dummy_x_one.transpose()), lt20_dummy_y_vals)


print("reg fit: ", w_vals)
print("lt 20 reg fit: ", lt20_w_vals)

w_tensor = torch.tensor(w_vals).reshape(1,-1,1)
print("w tensor: ", w_tensor)
lt20_w_tensor = torch.tensor(lt20_w_vals).reshape(1,-1,1)
print("w lt20 tensor: ", lt20_w_tensor)
# lt20_x1_vals_tensor = x1_vals[y_vals<20.0].reshape(1,-1,1)
# lt20_x2_vals_tensor = x2_vals[y_vals<20.0].reshape(1,-1,1)

dummy_w_tensor = torch.tensor(dummy_w_vals).reshape(1,-1,1)
lt20_dummy_w_tensor = torch.tensor(lt20_dummy_w_vals).reshape(1,-1,1)
# lt20_dummy_x1_vals_tensor = dummy_x1_vals[y_vals<20.0].reshape(1,-1,1)
# lt20_dummy_x2_vals_tensor = dummy_x2_vals[y_vals<20.0].reshape(1,-1,1)

lennn:  27
reg fit:  [1.98835287 3.14238058 0.06413647]
lt 20 reg fit:  [ 2.12311694  3.43333008 -1.03203902]
w tensor:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
w lt20 tensor:  tensor([[[ 2.1231],
         [ 3.4333],
         [-1.0320]]], dtype=torch.float64)


In [76]:
# Verifier/ data consumer side:
# Want to calculate regression of y over 
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = lt20_dummy_w_tensor, requires_grad = False)
        # self.new_size = nn.Parameter(data = new_dummy_size,requires_grad = False )
        self.new_Y = nn.Parameter(data = lt20_dummy_y_vals_tensor,requires_grad = False )
        self.new_X_one = nn.Parameter(data = lt20_dummy_x_one_tensor, requires_grad = False)
    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        # print("Y value: ", Y.size())
        # print("new x1: ", self.new_X_one.size())
        len_ratio =  self.new_Y.size()[1]/Y.size()[1]
        Y_where = torch.zeros(len_ratio*Y.size()[1], dtype=torch.float64).reshape(1,-1,1)
        Y_where[0]=self.new_Y[0]
        new_Y_cons = torch.sum((torch.abs(Y[Y<20.0].reshape(1,-1,1)-Y_where)<0.01*Y_where).double())==Y_where.size()[1] 
        # print("y cons: ", new_Y_cons)

        # bool_check = torch.zeros(self.new_X_one.size()[2]-1)
        # for i in range(self.new_X_one.size()[2]-1):
        #     print("hey: ", self.new_X_one[:,:,i:i+1].size())
        #     print("args: ", args[i][Y<20.0].reshape(1,-1,1).size())
        #     print("diff: ", torch.abs(args[i][Y<20.0].reshape(1,-1,1)-self.new_X_one[:,:,i:i+1] ))
        #     print("check::: ", torch.sum(torch.abs(args[i][Y<20.0].reshape(1,-1,1)-self.new_X_one[:,:,i:i+1] )<0.01*torch.abs(self.new_X_one[:,:,i:i+1])))
        #     bool_check[i] = torch.tensor(1)
        # print("bool check: ", bool_check)
        # bool_cons = torch.sum(bool_check) == self.new_X_one.size()[2]-1
        # print("bool cons sum: ", bool_cons)
        
        X_one_where = torch.zeros(3*len_ratio*Y.size()[1], dtype=torch.float64).reshape(1,Y_where.size()[1],-1)
        X_one_where[0] = self.new_X_one[0]

        X_where_T = torch.transpose(X_one_where, 1,2)
        # some expression of tolerance to error in the inference
        print("bool result: ",torch.sum(torch.abs(X_where_T @ X_one_where @ self.w - X_where_T @ Y_where)) < 0.01 * torch.sum(X_where_T @ Y_where))

        if (torch.sum(torch.abs(X_where_T @ X_one_where @ self.w - X_where_T @ Y_where)) < 0.01 * torch.sum(X_where_T @ Y_where)):
            value = torch.tensor(1)
        else:
            value = torch.tensor(0)
        print("value: ", value)
        return (
            torch.sum((X_where_T @ X_one_where @ self.w - X_where_T @ Y_where)) < 0.01 * torch.sum(X_where_T @ Y_where),
            self.w
        )
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_x1_vals_path, dummy_x2_vals_path, dummy_y_vals_path])

bool result:  tensor(True)
value:  tensor(1)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_53260/1290388594.py:39: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (torch.sum(torch.abs(X_where_T @ X_one_where @ self.w - X_where_T @ Y_where)) < 0.01 * torch.sum(X_where_T @ Y_where)):
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_53260/1290388594.py:40: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  value = torch.tensor(1)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarni

In [96]:
# prover calculates settings, send to verifier

theory_output = lt20_w_tensor
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = lt20_w_tensor, requires_grad = False)
        self.new_Y = nn.Parameter(data = lt20_y_vals_tensor,requires_grad = False )
        self.new_X_one = nn.Parameter(data = lt20_x_one_tensor, requires_grad = False)
    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        # print("Y value: ", Y.size())
        # print("new x1: ", self.new_X_one.size())
        len_ratio =  self.new_Y.size()[1]/Y.size()[1]
        Y_where = torch.zeros(len_ratio*Y.size()[1], dtype=torch.float64).reshape(1,-1,1)
        Y_where[0]=self.new_Y[0]
        new_Y_cons = torch.sum((torch.abs(Y[Y<20.0].reshape(1,-1,1)-Y_where)<0.01*torch.abs(Y_where)).double())==Y_where.size()[1] 
        print("y cons: ", new_Y_cons)



        X_one_where = torch.zeros(3*len_ratio*Y.size()[1], dtype=torch.float64).reshape(1,Y_where.size()[1],-1)
        X_one_where[0] = self.new_X_one[0]

        new_X_check = torch.zeros(self.new_X_one.size()[2]-1)
        for i in range(self.new_X_one.size()[2]-1):
            # print("hey: ", self.new_X_one[:,:,i:i+1].size())
            # print("args: ", args[i][Y<20.0].reshape(1,-1,1).size())
            new_X_check[i] = torch.sum(torch.abs(args[i][Y<20.0].reshape(1,-1,1)-X_one_where[:,:,i:i+1])<=0.01*torch.abs(X_one_where[:,:,i:i+1]))==self.new_X_one.size()[1]
        new_X_cons = torch.sum(new_X_check) == self.new_X_one.size()[2]-1
        print("bool sum cons: ", new_X_cons)
        
    
        X_where_T = torch.transpose(X_one_where, 1,2)

        X_one = torch.cat((*args[:-1], torch.ones_like(args[0][:, :, -1:])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)

        # bool_result = torch.sum(X_where_T @ Y_where)>3
        bool_result = torch.sum(X_where_T@Y_where)>3
        print("bool result: ",bool_result)

        # if (torch.sum(torch.abs(X_where_T @ X_one_where @ self.w - X_where_T @ Y_where)) < 0.01 * torch.sum(X_where_T @ Y_where)):
        #     value = torch.tensor(1)
        # else:
        #     value = torch.tensor(0)

        # if torch.logical_and(new_Y_cons, new_X_cons):
        #     final_val = torch.tensor(bool_result)
        # else: 
        #     final_val = torch.tensor(0)
        if torch.logical_and(new_X_cons, new_Y_cons):
            val = torch.sum((X_where_T @ Y_where)) >100
        else:
            val = torch.tensor(0)
        return (
            val,
            self.w
        )

        return (
            torch.sum((X_where_T @ X_one_where @ self.w - X_where_T @ Y_where)) < 0.01 * torch.sum(X_where_T @ Y_where),
            self.w
        )

prover_gen_settings([x1_vals_path, x2_vals_path, y_vals_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

Theory output:  tensor([[[ 2.1231],
         [ 3.4333],
         [-1.0320]]], dtype=torch.float64)
y cons:  tensor(True)
bool sum cons:  tensor(True)
bool result:  tensor(True)
==== Generate & Calibrate Setting ====


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_53260/1598429189.py:54: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.logical_and(new_X_cons, new_Y_cons):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:ConstantOfShape, node name: /ConstantOfShape): input typestr: T1, has unsupported type: tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/c

PanicException: index out of bounds: the len is 1 but the index is 1

In [7]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 3.1073861122131348 seconds
Theory output:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 2.0
witness result 2 : 3.0
witness result 3 : 0.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [0, 0, 0, 0]]], 'proof': '24468d86eb11b57e0f5992e33fe565dbd430a5121f2c7780745096bdb5a5aafb1346cba28a759e72bc91f71d73e534733d98cdee8cdba5d1d2dc48fe113a0c5f2d6c41fc95adcc977d0bb1d3f93b80cc66c91fd276dd7d5e90439d0a319f6469262b16a8b0713dd420ec8ff58fd8c4c25399fea8ac88dcf48560bceaec2263b20ed07e6a1d8df900344304ce59c81c443be8aaab509f023fd892db1bedb235bb0290f68444266bbc30446c02b4e4906aa6467c697effa713c1f22383e7a96d7f217faa8d568871bf75614aef4700e549b1a6e74cbf8031a2

In [8]:
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  3
prf instances:  [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [0, 0, 0, 0]]]
proof boolean:  1.0
proof result 1 : 2.0
proof result 2 : 3.0
proof result 3 : 0.0
verified
